In [ ]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(1, '../')

In [ ]:
# Uncomment the below commands if required to re-install the libraries
# %pip install faiss-cpu --quiet
# %pip install google.cloud --quiet
# %pip install cloud-sql-python-connector --quiet
# %pip install pg8000 --quiet
# %pip install langchain-google-vertexai

In [ ]:
import numpy as np
import faiss
from pandas import DataFrame
from datetime import datetime

In [ ]:
# PROJECT_ID = ! gcloud config get project
PROJECT_ID = 'sl-test-project'
LOCATION = 'us-central1'
PROJECT_ID

# Query and Embeddings in PostgreSQL

## Initialise

In [ ]:
%load_ext autoreload
%autoreload 2
from nl2sql_src.nl2sql_query_embeddings import PgSqlEmb

# Make sure you update the following variables in-line with your project
PGPROJ = "sl-test-project-353312"
PGLOCATION = 'us-central1'
PGINSTANCE = "test-nl2sql"
PGDB = "test-db"
PGUSER = "postgres"
PGPWD = "test-nl2sql"
# INDX_FILE = 'saved_index_pgdata' # optoinal, considered as default

pge = PgSqlEmb(PGPROJ, PGLOCATION, PGINSTANCE, PGDB, PGUSER, PGPWD)

## Create Table

In [ ]:
# Create Table in PostgresDB if not already existing
pge.create_table()

## Insert records to PostgreSQL and update index

In [ ]:
new_question = "How many unique products are in service?"
mapped_sql = "select count(distinct PRODUCT) from `sl-test-project-353312.EY.invoice_specific` "
pge.insert_row(new_question, mapped_sql)

## Retrieve data 

In [ ]:
# Retrieve the data from the PostgreDB and return a dataframe
def extract_data():
    tmp = pge.extract_data()
    df = DataFrame(tmp.fetchall())
    print(df)
    return df

df = extract_data()
df

## Search the VectorDB for closest matching queries

In [ ]:
# Search for the closest matching queries
new_query = 'What county has the highest number of beneficiaries enrolled in the WIC per capita?'
output_json = pge.search_matching_queries(new_query)
output_json

## Clear VectorDB and re-create VectorDB from all data in PostgreSQL

In [ ]:
pge.recreate_vectordb_index()

## Clear all Data in DB

In [ ]:
pge.empty_table()
df = extract_data()
df